In [1]:
import random
import warnings
import regex as re
import ast

In [2]:
warnings.filterwarnings('ignore')

## Step 1: Generate init -> goal pairs ##

In [ ]:
n=5

In [ ]:
def get_blocks(n):
  bag_of_blocks=[]
  for i in range(n):
    bag_of_blocks.append(i+1)
    
  return bag_of_blocks
print(get_blocks(n))

In [ ]:
from itertools import combinations
from itertools import permutations

def get_partitions(blocks):
    if not blocks:
        return [[]]
    
    partitions = []
    for i in range(1, len(blocks) + 1):
        for subset in combinations(blocks, i):
            remaining = [b for b in blocks if b not in subset]
            for partition in get_partitions(remaining):
                new_partition = [list(subset)] + list(partition)
                sorted_partition = tuple(sorted(tuple(sorted(group)) for group in new_partition))
                if sorted_partition not in partitions:
                    partitions.append(sorted_partition)
                    
    return partitions

blocks = get_blocks(n)
all_config=set()
all_configs = set(get_partitions(blocks))

print(all_configs)


In [ ]:
def get_all_permed(all_configs):
    w_permuations=[]
    for config in all_configs:
        #print(config)
        #config=((1,3),(2,))
        config_as_list=list(config)
        for idx,tower in enumerate(config_as_list):
                # print(idx,tower)
                if(len(tower)>1):
                    #print(idx,tower)
                    permutations_of_tower=tuple(permutations(tower))
                    modified_configs=[]

                    for element in permutations_of_tower:
                    
                        modified_config=config_as_list.copy()
                        modified_config[idx]=element
                        #äprint(f'Modified permutation:{modified_config}')
                        modified_configs.append(tuple(modified_config))
                    
                    for con in modified_configs:
                        w_permuations.append(tuple(con))
                    #print(perm_towers)
    return w_permuations
            
print(get_all_permed(all_configs))


In [ ]:
def get_all_states(n):

    blocks=get_blocks(n)
    all_configs = set(get_partitions(blocks))
    permutations=get_all_permed(all_configs)

    for element in permutations:
        if  element not in all_configs:
            all_configs.add(element)
    
    return all_configs

    

In [ ]:
all_states=get_all_states(n)
len(all_states)

In [ ]:
print(all_states)

In [ ]:
colours=['Red','Blue','Green','Yellow','Pink',
        'Orange','Purple','Brown','White','Violet',
        'Indigo','Ochre','Black','Magenta','Crimson',
        'Turquoise','Periwinkle','Grey','Emerald']

In [ ]:
#trial-run
selected_colours=random.sample(colours,n)
selected_colours

In [ ]:
#making init -> goal state pairs
import pandas as pd
def make_init_goal_lists(all_states):
    init=[]
    goal=[]
    list_all_states=list(all_states)
    for idx1,state_1 in enumerate(list_all_states):
        #print(idx1)
        for idx2, state_2 in enumerate(list_all_states):
            if idx1 != idx2:
                #print(f'works! {idx1}, {idx2}')
                init.append(list_all_states[idx1])
                goal.append(list_all_states[idx2])
    return init,goal

#trial-run
init,goal=make_init_goal_lists(all_states)                
for i in range(len(init)):
    print(f'For {n} blocks: Initial {init[i]} -> Goal {goal[i]}')


In [ ]:
d={'init':init,'goal':goal}
problems=pd.DataFrame(data=d)

In [ ]:
problems

In [ ]:
#shuffling problems to avoid clumps of same config across dataset splits
problems=problems.sample(frac=1).reset_index(drop=True)
problems

In [ ]:
train=problems[100000:162000].copy()
test=problems[162000:179000].copy()
val=problems[179000:].copy()
print(f"Size of splits: \nTrain: {len(train['init'])} \nValidation: {len(val['init'])}\nTest: {len(test['init'])}\n\nTOTAL:{len(val['init'])+len(test['init'])+len(train['init'])}")

In [ ]:
train.reset_index(drop=True,inplace=True)
train

In [ ]:
val.reset_index(drop=True,inplace=True)
val

In [ ]:
test.reset_index(drop=True,inplace=True)
test

In [ ]:
def fill_colour(state,colour_map):
    #debug line
    #print(f"Type received: {type(state)} - Value: {state}")
    # ((1,), (2,))
    return tuple(
        tuple(colour_map.get(block, block) for block in tower)
        for tower in state
    )

In [ ]:
fixcolour_set1=['red','blue','green','yellow','pink','grey']
fixcolour_set2=['orange','purple','brown','white','violet','teal']

In [ ]:
#varcolour_set1=random.sample(fixcolour_set1,n)
#varcolour_set2=random.sample(fixcolour_set2,n)
#print(f'Set 1:{varcolour_set1} \nSet 2: {varcolour_set2}')

In [ ]:
#colours=[varcolour_set1,varcolour_set2]
colours

In [ ]:
def get_col_map(colours):

    keys=colours[0]+colours[1]
    values=colours[1]+colours[0]
    colour_map=dict(zip(keys,values))
    return colour_map

In [ ]:
colour_map=get_col_map(colours)
colour_map

In [ ]:
#colours each row by randomly selecting one of two colour sets for each row
def get_row_colored(df,i):
    selected_colours={}
    idx_set=random.sample([0,1],1)[0]
    values=colours[idx_set]
    keys=[i for i in range(1,n+1)]
    selected_colours=dict(zip(keys,values))
    print(f'Selected colours for this round: {selected_colours}')
    
    df['init'][i]=fill_colour(df['init'][i],selected_colours)
    df['goal'][i]=fill_colour(df['goal'][i],selected_colours)

In [ ]:
#colours the entire dataframe row-by-row
def color_me_pretty(df):
    for i in range(len(df['init'])):
        print('-----------------------------------------------------------------\n\n')
        print(f"Getting row {i} coloured, {df['init'][i]} -> {df['goal'][i]}\n")
        get_row_colored(df,i=i)
        print(f"\nAfter colouring row {i} , {df['init'][i]} -> {df['goal'][i]}\n")
    

In [ ]:
color_me_pretty(train)

In [ ]:
train

In [ ]:
#train.to_csv(f'../data/train_{n}_blocks')

In [ ]:
#train.to_csv(f'../data/train_{n}_blocks_contn')

In [ ]:
color_me_pretty(test)

In [ ]:
test

In [ ]:
#test.to_csv(f'../data/test_{n}_blocks')

In [ ]:
test.to_csv(f'../data/test_{n}_blocks_contn')

In [ ]:
color_me_pretty(val)

In [ ]:
val

In [ ]:
#val.to_csv(f'../data/val_{n}_blocks')

In [ ]:
val.to_csv(f'../data/val_{n}_blocks_contn')

In [ ]:
test

In [ ]:
#contaminated test set
contaminated_test=test[:3793].copy()
contaminated_test

In [ ]:
def swap_colours(state,colour_map):
    return tuple(
        tuple(colour_map[block] if block in colour_map else block for block in tower) for tower in state)

In [ ]:
def contiminate(df,colour_map):
    for i in range(len(df['init'])):
        
        print('-----------------------------------------------------------------\n\n')
        print(f'Colour map is: {colour_map}\n\n')
        print(f"Swapping colours of row {i}:, {df['init'][i]} -> {df['goal'][i]}\n")
        
        #swap colour for each row
        df['init'][i]=swap_colours(df['init'][i],colour_map)
        df['goal'][i]=swap_colours(df['goal'][i],colour_map)

        print(f"\nAfter swapping colours of row {i} , {df['init'][i]} -> {df['goal'][i]}\n")


In [ ]:
colour_map=get_col_map(colours)
contiminate(contaminated_test,colour_map)

In [ ]:
contaminated_test

In [ ]:
#contaminated_test.to_csv(f'../data/contaminated_test_{n}_blocks')

## Step 2: Generation of own dataset akin to planbench dataset ##

In [3]:
def create_ic(state):
    state = ast.literal_eval(state)
    init=""
    clear=[]
    print(f'State in create_ic {state}' )
    for tower in state:
        tower_l=list(tower)
        # print(f'Tower: {tower_l}')

        for i,block in enumerate(tower_l):
            
            if len(tower_l) > 1:
                # print(f'Height of tower : {len(tower_l)}')
                # print(f'Index of block: {i}')

                #lowermost block in the tower
                if i == 0:
                    # print(f' the {block} block is on the table,')
                    init=init+f' the {str(block).lower()} block is on the table,'

                #highest block in the tower
                elif i == len(tower_l)-1:
                    # print(f' the {block} block is on top of the {tower_l[i-1]} block,')
                    init=init+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'
                    # print(f' the {block} block is clear,')
                    clear.append({str(block).lower()})

                #everything else
                else:
                    # print(f' the {block} block is on top of the {tower_l[i-1]} block,')
                    init=init+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'


            else:
                # print(f'Height of tower : {len(tower)}')
                # print(f' the {str(block)} block is on the table,')
                init=init+f' the {str(block).lower()} block is on the table,'
                clear.append({str(block).lower()})

    #print('#######',init)

    clear_str=""
    for element in clear:
        for block in element:
            clear_str=clear_str+f'the {block.lower()} block is clear, '

    init=clear_str+' the hand is empty,'+init
    init=init[:-1] #remove the tail ,

    return init

In [4]:
#i_state="(('Purple', 'Violet', 'Teal', 'Orange', 'Brown'),)"
#i_state= "(('Teal', 'White'), ('Purple',))"
i_state="(('Grey',), ('Pink',), ('Blue', 'Yellow'), ('Green',))"
init=create_ic(i_state)
print(init)

State in create_ic (('Grey',), ('Pink',), ('Blue', 'Yellow'), ('Green',))
the grey block is clear, the pink block is clear, the yellow block is clear, the green block is clear,  the hand is empty, the grey block is on the table, the pink block is on the table, the blue block is on the table, the yellow block is on top of the blue block, the green block is on the table


In [5]:
ic=init.split(',')

In [6]:
ic

['the grey block is clear',
 ' the pink block is clear',
 ' the yellow block is clear',
 ' the green block is clear',
 '  the hand is empty',
 ' the grey block is on the table',
 ' the pink block is on the table',
 ' the blue block is on the table',
 ' the yellow block is on top of the blue block',
 ' the green block is on the table']

In [7]:
def create_goal(state):
    state = ast.literal_eval(state)
    goal=""
    #all_clear=True
    print(f'State in create_goal {state}' )
    for tower in state:
        tower_l=list(tower)
        #print(tower_l)
    
        for i,block in enumerate(tower_l):
            
            if len(tower_l) > 1:

                all_clear=False #found a tower with multiple blocks
                
                #lowermost block in the tower
                if i == 0:
                    # print(f' the {block} block is on the table,')
                    goal=goal+f' the {str(block).lower()} block is on the table,'

                if i>0:
                    #print(f' the {str(block)} block is on top of the {str(tower_l[i-1])} block,')
                    goal=goal+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'

            else :
                goal=goal+f' the {str(block).lower()} block is on the table,'
                
    
    return goal[:-1]

In [8]:
g_state="(('Blue', 'Green', 'Grey'), ('Pink',), ('Yellow',))"
# g_state="(('White',), ('Teal',), ('Purple',))"
goal=create_goal(g_state)
goal

State in create_goal (('Blue', 'Green', 'Grey'), ('Pink',), ('Yellow',))


' the blue block is on the table, the green block is on top of the blue block, the grey block is on top of the green block, the pink block is on the table, the yellow block is on the table'

In [9]:
gs=goal.split(',')
gs

[' the blue block is on the table',
 ' the green block is on top of the blue block',
 ' the grey block is on top of the green block',
 ' the pink block is on the table',
 ' the yellow block is on the table']

In [11]:
import unifiedplanning_blocksworld as ubs
import planbench as pb
from importlib import reload

In [12]:
reload(pb)
reload(ubs)

<module 'unifiedplanning_blocksworld' from '/srv/chawak/planning-with-llms/src/unifiedplanning_blocksworld.py'>

In [13]:
problem=ubs.BlocksworldProblem()
#init,goal=pb.parse_problem(pbstring)
pb.parse_planbench_initial_condition(problem, init)
pb.parse_planbench_goal_state(problem, goal)
print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')



Blocksworld Problem Initial Values:{clear(grey): true, clear(pink): true, clear(yellow): true, clear(green): true, hand_empty: true, on_table(grey): true, on_table(pink): true, on_table(blue): true, on(yellow, blue): true, on_table(green): true, on(grey, grey): false, on(pink, grey): false, on(yellow, grey): false, on(green, grey): false, on(blue, grey): false, on(grey, pink): false, on(pink, pink): false, on(yellow, pink): false, on(green, pink): false, on(blue, pink): false, on(grey, yellow): false, on(pink, yellow): false, on(yellow, yellow): false, on(green, yellow): false, on(blue, yellow): false, on(grey, green): false, on(pink, green): false, on(yellow, green): false, on(green, green): false, on(blue, green): false, on(grey, blue): false, on(pink, blue): false, on(green, blue): false, on(blue, blue): false, clear(blue): false, on_table(yellow): false, holding(grey): false, holding(pink): false, holding(yellow): false, holding(green): false, holding(blue): false}

Blocksworld P

In [14]:
problem.blocks

{'grey': grey, 'pink': pink, 'yellow': yellow, 'green': green, 'blue': blue}

In [15]:
plan=problem.generate_plan()

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 132 of `/srv/chawak/planning-with-llms/src/unifiedplanning_blocksworld.py`, you are using the following planning engine:
  * Engine name: pyperplan
  * Developers:  Albert-Ludwigs-Universität Freiburg (Yusra Alkhazraji, Matthias Frorath, Markus Grützner, Malte Helmert, Thomas Liebetraut, Robert Mattmüller, Manuela Ortlieb, Jendrik Seipp, Tobias Springenberg, Philip Stahl, Jan Wülfing)
  * Description: Pyperplan is a lightweight STRIPS planner written in Python.



In [16]:
planstring=str(plan.plan)
plan_strings=planstring.split('\n')
plan_strings=[plan_string.strip() for plan_string in plan_strings]
plan_strings=plan_strings[1:]

In [17]:
plan_strings

['unstack(yellow, blue)',
 'put_down(yellow)',
 'pick_up(green)',
 'stack(green, blue)',
 'pick_up(grey)',
 'stack(grey, green)']

In [18]:
def form_action_string(action):

    if 'unstack' in action:
        m=re.search(r"unstack\((\w+), (\w+)\)",action)
        return f"unstack the {m.group(1)} block from on top of the {m.group(2)} block"
    
    elif 'pick_up' in action:
        m=re.search(r"pick_up\((\w+)",action)
        return f"pick up the {m.group(1)} block"
    
    elif 'put_down' in action:
        m=re.search(r"put_down\((\w+)",action)
        return f"put down the {m.group(1)} block"
    
    elif 'stack' in action:
        m=re.search(r"stack\((\w+), (\w+)\)",action)
        return f"stack the {m.group(1)} block on top of the {m.group(2)} block"

In [19]:
plan_strings = [form_action_string(plan_string) for plan_string in plan_strings] 
plan_strings = ['[PLAN]'] + plan_strings + ['[PLAN END]']

In [20]:
plan_strings

['[PLAN]',
 'unstack the yellow block from on top of the blue block',
 'put down the yellow block',
 'pick up the green block',
 'stack the green block on top of the blue block',
 'pick up the grey block',
 'stack the grey block on top of the green block',
 '[PLAN END]']

In [56]:
#load train data
n=4
import pandas as pd
trainData=pd.read_csv(f'../data/train_{n}_blocks')
trainData.drop(columns='Unnamed: 0',inplace=True)

In [57]:
trainData

,init,goal
0,"(('teal', 'orange'), ('white', 'purple'))","(('purple', 'teal'), ('orange',), ('white',))"
1,"(('red', 'pink', 'yellow', 'blue'),)","(('blue', 'yellow'), ('pink', 'red'))"
2,"(('teal',), ('purple', 'white'), ('orange',))","(('orange', 'teal', 'white', 'purple'),)"
3,"(('yellow', 'red', 'pink', 'blue'),)","(('blue', 'yellow'), ('red', 'pink'))"
4,"(('white', 'purple', 'orange', 'teal'),)","(('teal', 'purple'), ('white', 'orange'))"
...,...,...
3376,"(('yellow', 'pink', 'blue', 'red'),)","(('pink', 'blue', 'red', 'yellow'),)"
3377,"(('teal', 'purple', 'orange'), ('white',))","(('teal', 'white'), ('purple', 'orange'))"
3378,"(('orange', 'purple', 'teal'), ('white',))","(('purple', 'teal', 'white', 'orange'),)"
3379,"(('orange', 'teal', 'purple', 'white'),)","(('teal', 'purple', 'white'), ('orange',))"


#### 2. a. FOR TRAIN DATASET ####

In [25]:
#FOR TRAIN DATA: extract demonstrations
demo_rows=340
demo=trainData[:demo_rows].copy()
demo.reset_index(drop=True,inplace=True)
demo

,init,goal
0,"(('teal', 'orange'), ('white', 'purple'))","(('purple', 'teal'), ('orange',), ('white',))"
1,"(('red', 'pink', 'yellow', 'blue'),)","(('blue', 'yellow'), ('pink', 'red'))"
2,"(('teal',), ('purple', 'white'), ('orange',))","(('orange', 'teal', 'white', 'purple'),)"
3,"(('yellow', 'red', 'pink', 'blue'),)","(('blue', 'yellow'), ('red', 'pink'))"
4,"(('white', 'purple', 'orange', 'teal'),)","(('teal', 'purple'), ('white', 'orange'))"
...,...,...
335,"(('blue', 'pink', 'red', 'yellow'),)","(('blue', 'pink'), ('yellow', 'red'))"
336,"(('yellow', 'blue'), ('red',), ('pink',))","(('blue', 'red', 'pink', 'yellow'),)"
337,"(('blue',), ('pink', 'red'), ('yellow',))","(('pink', 'yellow', 'blue', 'red'),)"
338,"(('blue',), ('red',), ('yellow', 'pink'))","(('pink', 'blue', 'red'), ('yellow',))"


In [26]:
#FOR TRAIN DATA: REMOVE extract demo rows from training data
rows_to_drop=list(range(0,demo_rows))
trainData=trainData.drop(rows_to_drop, axis=0).reset_index(drop=True)
trainData

,init,goal
0,"(('purple', 'white', 'teal'), ('orange',))","(('orange', 'purple', 'teal'), ('white',))"
1,"(('purple', 'teal'), ('orange',), ('white',))","(('white', 'teal', 'orange', 'purple'),)"
2,"(('red', 'yellow', 'blue'), ('pink',))","(('blue', 'red', 'yellow'), ('pink',))"
3,"(('red', 'pink', 'blue', 'yellow'),)","(('pink', 'red', 'blue'), ('yellow',))"
4,"(('red', 'blue', 'yellow'), ('pink',))","(('blue', 'pink'), ('yellow', 'red'))"
...,...,...
3036,"(('yellow', 'pink', 'blue', 'red'),)","(('pink', 'blue', 'red', 'yellow'),)"
3037,"(('teal', 'purple', 'orange'), ('white',))","(('teal', 'white'), ('purple', 'orange'))"
3038,"(('orange', 'purple', 'teal'), ('white',))","(('purple', 'teal', 'white', 'orange'),)"
3039,"(('orange', 'teal', 'purple', 'white'),)","(('teal', 'purple', 'white'), ('orange',))"


#### 2. b. FOR VAL/TEST DATA: ####

In [55]:
#load dataset to be modified
df='test'
df=pd.read_csv(f'../data/{df}_{n}_blocks')
df.drop(columns='Unnamed: 0',inplace=True)
df

,init,goal
0,"(('pink', 'blue'), ('red',), ('yellow',))","(('blue',), ('red', 'pink', 'yellow'))"
1,"(('blue', 'red'), ('yellow', 'pink'))","(('blue', 'pink', 'red', 'yellow'),)"
2,"(('blue', 'pink', 'red'), ('yellow',))","(('blue', 'red', 'pink'), ('yellow',))"
3,"(('teal', 'white', 'purple', 'orange'),)","(('teal', 'white'), ('purple', 'orange'))"
4,"(('yellow', 'blue'), ('red', 'pink'))","(('pink', 'yellow', 'blue'), ('red',))"
...,...,...
961,"(('pink', 'red', 'blue'), ('yellow',))","(('pink', 'yellow', 'blue'), ('red',))"
962,"(('white', 'teal', 'purple'), ('orange',))","(('orange', 'white', 'teal'), ('purple',))"
963,"(('teal',), ('purple',), ('orange', 'white'))","(('white', 'teal', 'orange', 'purple'),)"
964,"(('blue', 'yellow', 'pink', 'red'),)","(('red', 'yellow', 'blue'), ('pink',))"


In [58]:
#shuffling problems to avoid clumps of same demonstrations config across dataset splits
trainData=trainData.sample(frac=1).reset_index(drop=True)
trainData

,init,goal
0,"(('white', 'teal'), ('purple',), ('orange',))","(('teal', 'orange'), ('white', 'purple'))"
1,"(('blue',), ('red',), ('yellow', 'pink'))","(('blue',), ('red', 'pink'), ('yellow',))"
2,"(('red', 'blue', 'yellow'), ('pink',))","(('blue',), ('pink', 'yellow', 'red'))"
3,"(('purple', 'teal'), ('orange',), ('white',))","(('teal', 'white', 'orange'), ('purple',))"
4,"(('pink', 'blue'), ('red', 'yellow'))","(('pink', 'blue', 'red', 'yellow'),)"
...,...,...
3376,"(('yellow', 'blue', 'red'), ('pink',))","(('red', 'pink', 'blue'), ('yellow',))"
3377,"(('orange', 'teal', 'white'), ('purple',))","(('teal', 'white'), ('purple',), ('orange',))"
3378,"(('blue',), ('pink', 'yellow', 'red'))","(('pink', 'blue'), ('red',), ('yellow',))"
3379,"(('blue',), ('red', 'yellow', 'pink'))","(('red', 'pink', 'blue', 'yellow'),)"


In [59]:
#extract demo-rows
demo_rows=len(df['init'])
demo=trainData[:demo_rows].copy()
demo.reset_index(drop=True,inplace=True)
demo

,init,goal
0,"(('white', 'teal'), ('purple',), ('orange',))","(('teal', 'orange'), ('white', 'purple'))"
1,"(('blue',), ('red',), ('yellow', 'pink'))","(('blue',), ('red', 'pink'), ('yellow',))"
2,"(('red', 'blue', 'yellow'), ('pink',))","(('blue',), ('pink', 'yellow', 'red'))"
3,"(('purple', 'teal'), ('orange',), ('white',))","(('teal', 'white', 'orange'), ('purple',))"
4,"(('pink', 'blue'), ('red', 'yellow'))","(('pink', 'blue', 'red', 'yellow'),)"
...,...,...
961,"(('orange', 'teal', 'white', 'purple'),)","(('teal', 'purple', 'orange', 'white'),)"
962,"(('blue',), ('red', 'yellow', 'pink'))","(('blue',), ('red', 'yellow'), ('pink',))"
963,"(('pink', 'blue'), ('red',), ('yellow',))","(('pink', 'blue', 'red'), ('yellow',))"
964,"(('teal', 'purple'), ('orange', 'white'))","(('orange', 'purple', 'teal', 'white'),)"


### 3. Generate & add demonstrations ###

In [60]:
def create_ubs_object(demo_row):
    problem=ubs.BlocksworldProblem()
    init=create_ic(demo_row['init'])
    goal=create_goal(demo_row['goal'])

    print(f'parsed initial conditions: {init}')
    print(f'parsed goal conditions: {goal}')

    pb.parse_planbench_initial_condition(problem, init)
    pb.parse_planbench_goal_state(problem, goal)

    print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
    print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
    plan=problem.generate_plan()
    return init,goal,plan
    

In [61]:
def add_plans_to_demo(demo):
    
    demo['plan']=''

    for index, row in demo.iterrows():
        print('Demo row:',index, row[0],'->',row[1])
        init,goal,plan=create_ubs_object(row)

        demo.at[index, 'init']=init
        demo.at[index, 'goal']=goal

        #string from plan object
        plan_strings=str(plan.plan).strip().split('\n')
        plan_strings=plan_strings[1:]
        plan_strings = [form_action_string(plan_string) for plan_string in plan_strings] 
        plan_strings = ['[PLAN]'] + plan_strings + ['[PLAN END]']

        print(f'------PLAN STRINGS------ AFTER PROCESSING: {plan_strings}')
        processed_plan='\n'.join(plan_strings)
        demo.at[index, 'plan']=processed_plan
        
    

In [62]:
add_plans_to_demo(demo)

Demo row: 0 (('white', 'teal'), ('purple',), ('orange',)) -> (('teal', 'orange'), ('white', 'purple'))
State in create_ic (('white', 'teal'), ('purple',), ('orange',))
State in create_goal (('teal', 'orange'), ('white', 'purple'))
parsed initial conditions: the teal block is clear, the purple block is clear, the orange block is clear,  the hand is empty, the white block is on the table, the teal block is on top of the white block, the purple block is on the table, the orange block is on the table
parsed goal conditions:  the teal block is on the table, the orange block is on top of the teal block, the white block is on the table, the purple block is on top of the white block


Blocksworld Problem Initial Values:{clear(teal): true, clear(purple): true, clear(orange): true, hand_empty: true, on_table(white): true, on(teal, white): true, on_table(purple): true, on_table(orange): true, on(teal, teal): false, on(purple, teal): false, on(orange, teal): false, on(white, teal): false, on(teal,

In [63]:
demo

,init,goal,plan
0,"the teal block is clear, the purple block is c...","the teal block is on the table, the orange bl...",[PLAN]\nunstack the teal block from on top of ...
1,"the blue block is clear, the red block is clea...","the blue block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
2,"the yellow block is clear, the pink block is c...","the blue block is on the table, the pink bloc...",[PLAN]\nunstack the yellow block from on top o...
3,"the teal block is clear, the orange block is c...","the teal block is on the table, the white blo...",[PLAN]\npick up the orange block\nstack the or...
4,"the blue block is clear, the yellow block is c...","the pink block is on the table, the blue bloc...",[PLAN]\nunstack the yellow block from on top o...
...,...,...,...
961,"the purple block is clear, the hand is empty,...","the teal block is on the table, the purple bl...",[PLAN]\nunstack the purple block from on top o...
962,"the blue block is clear, the pink block is cle...","the blue block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
963,"the blue block is clear, the red block is clea...","the pink block is on the table, the blue bloc...",[PLAN]\npick up the red block\nstack the red b...
964,"the purple block is clear, the white block is ...","the orange block is on the table, the purple ...",[PLAN]\nunstack the purple block from on top o...


In [32]:
#!!!!!!!!! ONLY FOR TRAIN DATASET !!!!!!!!!
def sample_and_add(df,demo):
    df['demo_init']=""
    df['demo_goal']=""
    df['demo_plan']=""

    for i in range(len(df['init'])):

        # Sample a random row from the demo dataframe
        random_demo_row = demo.sample(n=1).iloc[0]

        # Define the index of the row in 'train' where you want to add the demo data
        val_index = i 

        # Assign values from demo to the corresponding columns in train
        df.loc[val_index, ['demo_init', 'demo_goal', 'demo_plan']] = random_demo_row[['init', 'goal', 'plan']].values
    return df

In [33]:
#trainData=sample_and_add(trainData,demo)

In [34]:
#trainData

,init,goal,demo_init,demo_goal,demo_plan
0,"(('purple', 'white', 'teal'), ('orange',))","(('orange', 'purple', 'teal'), ('white',))","the pink block is clear, the yellow block is c...","the red block is on the table, the blue block...",[PLAN]\npick up the yellow block\nstack the ye...
1,"(('purple', 'teal'), ('orange',), ('white',))","(('white', 'teal', 'orange', 'purple'),)","the yellow block is clear, the pink block is c...","the blue block is on the table, the red block...",[PLAN]\nunstack the yellow block from on top o...
2,"(('red', 'yellow', 'blue'), ('pink',))","(('blue', 'red', 'yellow'), ('pink',))","the blue block is clear, the yellow block is c...","the blue block is on the table, the yellow bl...",[PLAN]\nunstack the yellow block from on top o...
3,"(('red', 'pink', 'blue', 'yellow'),)","(('pink', 'red', 'blue'), ('yellow',))","the orange block is clear, the hand is empty,...","the teal block is on the table, the orange bl...",[PLAN]\nunstack the orange block from on top o...
4,"(('red', 'blue', 'yellow'), ('pink',))","(('blue', 'pink'), ('yellow', 'red'))","the purple block is clear, the white block is ...","the teal block is on the table, the orange bl...",[PLAN]\npick up the white block\nstack the whi...
...,...,...,...,...,...
3036,"(('yellow', 'pink', 'blue', 'red'),)","(('pink', 'blue', 'red', 'yellow'),)","the blue block is clear, the yellow block is c...","the blue block is on the table, the pink bloc...",[PLAN]\nunstack the blue block from on top of ...
3037,"(('teal', 'purple', 'orange'), ('white',))","(('teal', 'white'), ('purple', 'orange'))","the white block is clear, the purple block is ...","the teal block is on the table, the white blo...",[PLAN]\nunstack the white block from on top of...
3038,"(('orange', 'purple', 'teal'), ('white',))","(('purple', 'teal', 'white', 'orange'),)","the purple block is clear, the hand is empty,...","the teal block is on the table, the orange bl...",[PLAN]\nunstack the purple block from on top o...
3039,"(('orange', 'teal', 'purple', 'white'),)","(('teal', 'purple', 'white'), ('orange',))","the white block is clear, the orange block is ...","the white block is on the table, the teal blo...",[PLAN]\nunstack the white block from on top of...


In [35]:
trainData['init'][10]

"(('blue', 'red'), ('pink', 'yellow'))"

In [36]:
trainData['goal'][10]

"(('red', 'pink', 'blue'), ('yellow',))"

In [37]:
trainData['demo_init'][10]

'the blue block is clear, the red block is clear, the pink block is clear, the yellow block is clear,  the hand is empty, the blue block is on the table, the red block is on the table, the pink block is on the table, the yellow block is on the table'

In [38]:
trainData['demo_goal'][10]

' the yellow block is on the table, the blue block is on top of the yellow block, the red block is on the table, the pink block is on top of the red block'

In [39]:
print(trainData['demo_plan'][10])

[PLAN]
pick up the blue block
stack the blue block on top of the yellow block
pick up the pink block
stack the pink block on top of the red block
[PLAN END]


In [40]:
trainData.to_csv(f'../data/Oneshot_train_{n}_blocks')

In [64]:
def add_demo_to_df(df,demo):
    df['demo_init']=demo['init']
    df['demo_goal']=demo['goal']
    df['demo_plan']=demo['plan']
    return df

In [65]:
df=add_demo_to_df(df,demo)
df

,init,goal,demo_init,demo_goal,demo_plan
0,"(('pink', 'blue'), ('red',), ('yellow',))","(('blue',), ('red', 'pink', 'yellow'))","the teal block is clear, the purple block is c...","the teal block is on the table, the orange bl...",[PLAN]\nunstack the teal block from on top of ...
1,"(('blue', 'red'), ('yellow', 'pink'))","(('blue', 'pink', 'red', 'yellow'),)","the blue block is clear, the red block is clea...","the blue block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
2,"(('blue', 'pink', 'red'), ('yellow',))","(('blue', 'red', 'pink'), ('yellow',))","the yellow block is clear, the pink block is c...","the blue block is on the table, the pink bloc...",[PLAN]\nunstack the yellow block from on top o...
3,"(('teal', 'white', 'purple', 'orange'),)","(('teal', 'white'), ('purple', 'orange'))","the teal block is clear, the orange block is c...","the teal block is on the table, the white blo...",[PLAN]\npick up the orange block\nstack the or...
4,"(('yellow', 'blue'), ('red', 'pink'))","(('pink', 'yellow', 'blue'), ('red',))","the blue block is clear, the yellow block is c...","the pink block is on the table, the blue bloc...",[PLAN]\nunstack the yellow block from on top o...
...,...,...,...,...,...
961,"(('pink', 'red', 'blue'), ('yellow',))","(('pink', 'yellow', 'blue'), ('red',))","the purple block is clear, the hand is empty,...","the teal block is on the table, the purple bl...",[PLAN]\nunstack the purple block from on top o...
962,"(('white', 'teal', 'purple'), ('orange',))","(('orange', 'white', 'teal'), ('purple',))","the blue block is clear, the pink block is cle...","the blue block is on the table, the red block...",[PLAN]\nunstack the pink block from on top of ...
963,"(('teal',), ('purple',), ('orange', 'white'))","(('white', 'teal', 'orange', 'purple'),)","the blue block is clear, the red block is clea...","the pink block is on the table, the blue bloc...",[PLAN]\npick up the red block\nstack the red b...
964,"(('blue', 'yellow', 'pink', 'red'),)","(('red', 'yellow', 'blue'), ('pink',))","the purple block is clear, the white block is ...","the orange block is on the table, the purple ...",[PLAN]\nunstack the purple block from on top o...


In [66]:
df.to_csv(f'../data/Oneshot_test_{n}_blocks')

In [67]:
import pandas as pd
df=pd.read_csv(f'../data/Oneshot_test_{n}_blocks')

In [54]:
df.iloc[11]

Unnamed: 0                                                   11
init                   (('orange', 'teal', 'purple', 'white'),)
goal              (('teal', 'white'), ('purple',), ('orange',))
demo_init     the teal block is clear, the white block is cl...
demo_goal      the teal block is on the table, the orange bl...
demo_plan     [PLAN]\nunstack the teal block from on top of ...
Name: 11, dtype: object

In [68]:
df.iloc[11]

Unnamed: 0                                                   11
init                       (('red', 'blue', 'yellow', 'pink'),)
goal                     (('blue', 'red', 'pink'), ('yellow',))
demo_init     the yellow block is clear, the pink block is c...
demo_goal      the blue block is on the table, the red block...
demo_plan     [PLAN]\nunstack the pink block from on top of ...
Name: 11, dtype: object